In [7]:
# analysis.ipynb

import os
import sys
import yaml
import torch
import matplotlib.pyplot as plt

# 1. Ensure train.py (and its Configs) is importable
proj = os.getcwd()  # assume notebook is opened in FEDformer/
sys.path.insert(0, os.path.join(proj, "scripts"))

from train import Configs, load_yaml_config
from fedformer.models.FEDformer import Model
from sklearn.preprocessing import StandardScaler

# 2. Load config dict and wrap in Configs
cfg_dict = load_yaml_config()
configs = Configs(cfg_dict)

# 3. Rebuild your scaler (use the same params you saved in preprocess)
scaler = StandardScaler()
# scaler.mean_ and scaler.scale_ must match what you used in preprocess:
# e.g. scaler.mean_ = [mean_close]; scaler.scale_ = [std_close]

# 4. Instantiate and load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(configs).to(device)

# Confirm the exact path we’re loading from
state_dict_path = os.path.join(proj, "models", "fedformer.pth")
print("Loading weights from:", state_dict_path)

model.load_state_dict(torch.load(state_dict_path, map_location=device))
model.eval()

# 5. Grab one example batch from your data
data_path = os.path.join(proj, cfg_dict["data"]["tensor_path"])
X, Y = torch.load(data_path)
bx = X[:1].to(device)  # shape (1, seq_len, 1)
by = Y[:1].to(device)

# 6. Run inference
with torch.no_grad():
    # Create dummy time features as in train.py
    zeros_enc = torch.zeros_like(bx, device=device)
    zeros_dec = torch.zeros(1, configs.pred_len, zeros_enc.size(-1), device=device)
    pred = model(bx, zeros_enc, by, zeros_dec)  # (1, pred_len, 1)

# 7. Plot normalized predictions vs. truth
y_true = by.cpu().numpy().squeeze()
y_pred = pred.cpu().numpy().squeeze()

plt.figure(figsize=(10, 4))
plt.plot(range(len(y_true)), y_true, label="True")
plt.plot(range(len(y_true), len(y_true) + len(y_pred)), y_pred, label="Predicted")
plt.legend()
plt.title("FEDformer Forecast vs Ground Truth (normalized)")
plt.xlabel("Time step")
plt.ylabel("Normalized value")
plt.show()

base legendre
base legendre
base legendre
corss fourier correlation used!
corss fourier correlation used!
corss fourier correlation used!
corss fourier correlation used!
enc_modes: 32, dec_modes: 32
Loading weights from: /Users/mchildress/Active Code/ts_basics/FEDformer/models/fedformer.pth


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mchildress/Active Code/ts_basics/FEDformer/data/windows.pt'